<h1 style="color:Orange" align="center">Using Keras Tuner To Show The Best Result</h1>
<h2 style="color:Green" align="left">Reading Data</h2>

In [1]:
import pandas as pd
from PM8 import replacer, preprocessing
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("E:/Data_Science/python/datasets/training_set.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y


<h2 style="color:Green" align="left">Data Understanding</h2>

In [3]:
unique = {}
for i in df.columns:
    unique[i] = len(df[i].unique())
print("======Null Value Finder========")
print(df.isna().sum())
print("======Unique Columns Dictionary========")
print(unique)
print("======Describe Data========")
print(df.describe())

======Null Value Finder========
Loan_ID               0
Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64
======Unique Columns Dictionary========
{'Loan_ID': 614, 'Gender': 3, 'Married': 3, 'Dependents': 5, 'Education': 3, 'Self_Employed': 3, 'ApplicantIncome': 505, 'CoapplicantIncome': 287, 'LoanAmount': 204, 'Loan_Amount_Term': 11, 'Credit_History': 3, 'Property_Area': 3, 'Loan_Status': 2}
======Describe Data========
       ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count       612.000000         613.000000  592.000000         600.00000   
mean       5405.540850        1620.888940  146.412162         342.00000   
std        6118.914057        2928.624748   85.587325          65.12041   
min         150.000000    


<h2 style="color:Green" align="left">Replacing Null Values</h2>

In [4]:
from PM8 import replacer
replacer(df)

<h2 style="color:Green" align="left">Define X and Y</h2>

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
Y = df[["Loan_Status"]]
X = df.drop(labels=["Loan_ID","Loan_Status"],axis=1)

<h2 style="color:Green" align="left">Label Encoder</h2>

In [7]:
from sklearn.preprocessing import LabelEncoder
import warnings

le = LabelEncoder()
Y.Loan_Status = le.fit_transform(Y)

In [8]:
Xnew = preprocessing(X)

In [9]:
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [10]:
xtrain.shape

(491, 20)

In [11]:
xtrain.shape[1]

20

<h2 style="color:Green" align="left">Creating ANN Model Untuned</h2>

In [12]:
nn = Sequential()
nn.add(Dense(128,input_dim=xtrain.shape[1]))
nn.add(Dense(20))
nn.add(Dense(20))
nn.add(Dense(20))
nn.add(Dense(20))
nn.add(Dense(1,activation="sigmoid"))

In [13]:
nn.compile(loss="binary_crossentropy",metrics="accuracy")
nn.fit(xtrain,ytrain,epochs=50,validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 1s 20ms/step - loss: 0.5994 - accuracy: 0.7143 - val_loss: 0.4969 - val_accuracy: 0.8081
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 0.4759 - accuracy: 0.8061 - val_loss: 0.5203 - val_accuracy: 0.7980
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 0.4615 - accuracy: 0.8036 - val_loss: 0.5111 - val_accuracy: 0.8182
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 0.4627 - accuracy: 0.8061 - val_loss: 0.5171 - val_accuracy: 0.7879
Epoch 5/50
13/13 [==============================] - 0s 4ms/step - loss: 0.4584 - accuracy: 0.7959 - val_loss: 0.4974 - val_accuracy: 0.8182
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4662 - accuracy: 0.8036 - val_loss: 0.5236 - val_accuracy: 0.7879
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4599 - accuracy: 0.8087 - val_loss: 0.4931 - val_accuracy: 0.8081
Epoch 8/50
13/13 [=

<h2 style="color:Green" align="left">Using Keras Tuner</h2>


In [14]:
def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(128, input_dim=xtrain.shape[1]))
    # This is the line whch is check by use
    x = hp.Int("units", min_value = 32, max_value = 256, step = 32)
    nn.add(Dense(units=x))
    nn.add(Dense(1, activation="sigmoid"))
    nn.compile(loss="binary_crossentropy", metrics="accuracy")
    return nn

<h2 style="color:Green" align="left">Hyperband</h2>

In [15]:
es = EarlyStopping(monitor="val_loss",patience=25)

In [23]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder,
                    objective="val_accuracy",
                    max_epochs=10,
                    directory="/Users/user/Desktop/python/datasets/Kt_nn_N_n2",
                    project_name="keras_tuner1")

In [20]:
tuner.search(xtrain, ytrain,epochs=100, validation_split=0.2)

Trial 8 Complete [00h 00m 01s]
val_accuracy: 0.808080792427063

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


<h2 style="color:Green" align="left">Using Call Back To stop iteration if it does not get any new increment</h2>

In [21]:

es = EarlyStopping(monitor="val_loss",patience=25)

In [24]:
tuner.search(xtrain, ytrain,epochs=50, validation_split=0.2, callbacks=[es])

Trial 8 Complete [00h 00m 01s]
val_accuracy: 0.7777777910232544

Best val_accuracy So Far: 0.8282828330993652
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


In [25]:
layers_size = tuner.get_best_hyperparameters(num_trials=1)[0].get("units")
layers_size

224

<h2 style="color:Green" align="left">Finally Used Tuned Value</h2>

In [26]:
nn = Sequential()
nn.add(Dense(layers_size,input_dim=xtrain.shape[1]))
nn.add(Dense(layers_size))
nn.add(Dense(layers_size))
nn.add(Dense(layers_size))
nn.add(Dense(1,activation="sigmoid"))
nn.compile(loss="binary_crossentropy",metrics="accuracy")
nn.fit(xtrain,ytrain,epochs=50,validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 1s 17ms/step - loss: 0.6101 - accuracy: 0.7296 - val_loss: 0.5992 - val_accuracy: 0.7677
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 0.5294 - accuracy: 0.7832 - val_loss: 0.5302 - val_accuracy: 0.8081
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4906 - accuracy: 0.7985 - val_loss: 0.5205 - val_accuracy: 0.7879
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4887 - accuracy: 0.8061 - val_loss: 0.5521 - val_accuracy: 0.7879
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4783 - accuracy: 0.7934 - val_loss: 0.5861 - val_accuracy: 0.8081
Epoch 6/50
13/13 [==============================] - 0s 5ms/step - loss: 0.4782 - accuracy: 0.8138 - val_loss: 0.5147 - val_accuracy: 0.7980
Epoch 7/50
13/13 [==============================] - 0s 6ms/step - loss: 0.4747 - accuracy: 0.7908 - val_loss: 0.6323 - val_accuracy: 0.7879
Epoch 8/50
13/13 [=